# Building an image retrieval system with deep features

In [1]:
import graphlab

# Load the CIFAR-10 dataset
We will use a popular benchmark dataset in computer vision called CIFAR-10.
(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)
This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [18]:
image_train = graphlab.SFrame("/home/aunja/Downloads/image_train_data.csv")
image_test = graphlab.SFrame("/home/aunja/Downloads/image_test_data.csv")

Finished parsing file /home/aunja/Downloads/image_train_data.csv

Parsing completed. Parsed 100 lines in 2.60705 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,array,array]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1943 lines. Lines per second: 597.928

Finished parsing file /home/aunja/Downloads/image_train_data.csv

...

Parsing completed. Parsed 100 lines in 2.63085 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,array,array]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1940 lines. Lines per second: 601.948

Finished parsing file /home/aunja/Downloads/image_test_data.csv

Parsing completed. Parsed 4000 lines in 4.6472 secs.

# Computing deep features for our images
The two lines below allow us to compute deep features. This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded.
(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [3]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242872, 1.09545, 0.0,0.39363, 0.0, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525088, 0.0, 0.0, 0.0,0.0, 0.0, 9.94829, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566016, 0.0, 0.0, 0.0,0.0, 0.0, 9.9972, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.1298, 0.0, 0.0,0.778194, 0.0, 0.758051, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71787, 0.0, 0.0, 0.0,0.0, 0.0, 9.33936, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57819, 0.0, 0.0, 0.0,0.0, 0.0, 9.00632, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.220678, 0.0,0.0, 0.0, 8.58053, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.237535, 0.0, 0.0,0.0, 0.0, 9.9908, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57379, 0.0, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658936, 0.0, 0.0, 0.0,0.0, 0.0, 9.93748, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Train a nearest-neighbors model for retrieving images using deep features
We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [4]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

# Use image retrieval model with deep features to find similar images
Let's find similar images to this cat picture.

In [5]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [6]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 139.35ms     |

| Done         |         | 100         | 520.86ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.940312418,2
0,39777,38.4634892031,3
0,36870,39.7559665724,4
0,41734,39.7865972971,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [7]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [8]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 47.285ms     |

| Done         |         | 100         | 494.009ms    |

+--------------+---------+-------------+--------------+

In [9]:
cat_neighbors['image'].show()

Very cool results showing similar cats.
# Finding similar images to a car

In [10]:
car = image_train[8:9]
car['image'].show()

In [11]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 46.432ms     |

| Done         |         | 100         | 454.257ms    |

+--------------+---------+-------------+--------------+

# Just for fun, let's create a lambda to find and show nearest neighbor images

In [12]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [13]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 49.986ms     |

| Done         |         | 100         | 504.787ms    |

+--------------+---------+-------------+--------------+

In [14]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 61.333ms     |

| Done         |         | 100         | 488.276ms    |

+--------------+---------+-------------+--------------+

# 1-What’s the least common category in the training data?

In [15]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


# Answer is bird

# 2-Of the images below, which is the nearest ‘cat’ labeled image in the training data to the the first image in the test data (image_test[0:1])?

Similarly to the image retrieval notebook you downloaded, you are going to create a nearest neighbor model using the 'deep_features' as the features, but this time create one such model for each category, using the corresponding subset of the training_data. You can call the model with the ‘dog’ data the dog_model, the one with the ‘cat’ data the cat_model, as so on.

In [26]:
dog_train=image_train[image_train['label']=='dog']
cat_train=image_train[image_train['label']=='cat']
bird_train=image_train[image_train['label']=='bird']
automobile_train=image_train[image_train['label']=='automobile']


In [16]:
dog_model = graphlab.nearest_neighbors.create(image_train.filter_by("dog","label"), features=['deep_features'], label='id')
cat_model = graphlab.nearest_neighbors.create(image_train.filter_by("cat","label"), features=['deep_features'], label='id')
auto_model = graphlab.nearest_neighbors.create(image_train.filter_by("automobile","label"), features=['deep_features'], label='id')
bird_model = graphlab.nearest_neighbors.create(image_train.filter_by("bird","label"), features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Now we have a nearest neighbors model that can find the nearest ‘dog’ to any image you give it, the dog_model; one that can find the nearest ‘cat’, the cat_model; and so on.
Using these models, answer the following questions. The cat image below is the first in the test data:
We can access this image, similarly to what we did in the iPython notebooks above, with this command:
image_test[0:1]

In [28]:
mycat=image_test[0:1]
image_test[0:1]['image'].show()

In [31]:
mycat_neighbors = get_images_from_ids(cat_model.query(mycat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 58.345ms     |

| Done         |         | 100         | 236.231ms    |

+--------------+---------+-------------+--------------+

In [32]:
mycat_neighbors['image'].show()

In [33]:
mycat_neighbors['id']

dtype: int
Rows: 5
[331, 16289, 25713, 32139, 45646]

In [34]:
mycat_neighbors

id,image,label,deep_features,image_array
331,Height: 32 Width: 32,cat,"[0.0, 0.0, 0.510964, 0.0,0.0, 0.0, 11.2724, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ..."
16289,Height: 32 Width: 32,cat,"[0.964288, 0.0, 0.0, 0.0,1.12516, 0.0, 9.31215, ...","[215.0, 219.0, 231.0,215.0, 219.0, 232.0, ..."
25713,Height: 32 Width: 32,cat,"[0.536971, 0.0, 0.0,0.0894459, 0.236474, ...","[228.0, 222.0, 236.0,224.0, 213.0, 222.0, ..."
32139,Height: 32 Width: 32,cat,"[1.29409, 0.0, 0.0,0.5138, 0.106392, ...","[217.0, 220.0, 205.0,221.0, 227.0, 218.0, ..."
45646,Height: 32 Width: 32,cat,"[0.983678, 0.0, 0.0, 0.0,0.0, 0.192085, 9.89629, ...","[51.0, 42.0, 26.0, 56.0,47.0, 31.0, 59.0, 50.0, ..."


In [36]:
cat_model.query(mycat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 93.319ms     |

| Done         |         | 100         | 274.478ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.6237223947,1
0,45646,36.0068799499,2
0,32139,36.5200809583,3
0,25713,36.7548479811,4
0,331,36.8731153276,5


# 3-Of the images below, which is the nearest ‘dog’ labeled image in the training data to the the first image in the test data (image_test[0:1])?

In [38]:
dog_model.query(mycat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 96.756ms     |

| Done         |         | 100         | 251.107ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.464261499,1
0,13387,37.5666834651,2
0,35867,37.6047270632,3
0,44603,37.7065587528,4
0,6094,38.5113288876,5


For the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘cat’ in the training data (similarly to what you did in the previous question). Save this result.
Similarly, for the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘dog’ in the training data (similarly to what you did in the previous question). Save this result.
On average, is the first image in the test data closer to its 5 nearest neighbors in the ‘cat’ data or in the ‘dog’ data? (In a later course, we will see that this is an example of what is called a k-nearest neighbors classifier, where we use the label of neighboring points to predict the label of a test point.)

In [66]:
get_images_from_ids(dog_model.query(image_test[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 48.84ms      |

| Done         |         | 100         | 244.746ms    |

+--------------+---------+-------------+--------------+

# 4-For the first image in the test data, in what range is the mean distance between this image and its 5 nearest neighbors that were labeled ‘cat’ in the training data?

In [40]:
cat_model.query(mycat)['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 51.902ms     |

| Done         |         | 100         | 273.545ms    |

+--------------+---------+-------------+--------------+

36.15572932231883

# Answer is 36.155

In [42]:
get_images_from_ids(dog_model.query(mycat))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 52.522ms     |

| Done         |         | 100         | 245.9ms      |

+--------------+---------+-------------+--------------+

In [44]:
cat_model.query(mycat)['distance']

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 43.516ms     |

| Done         |         | 100         | 266.119ms    |

+--------------+---------+-------------+--------------+

dtype: float
Rows: 5
[34.62372239472129, 36.006879949937115, 36.52008095832401, 36.75484798105772, 36.87311532755401]

# 5-For the first image in the test data, in what range is the mean distance between this image and its 5 nearest neighbors that were labeled ‘dog’ in the training data?

In [45]:
dog_model.query(mycat)['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 45.62ms      |

| Done         |         | 100         | 240.006ms    |

+--------------+---------+-------------+--------------+

37.7707119335295

Spliting test data by label: Above, you split the train data SFrame into one SFrame for images labeled ‘dog’, another for those labeled ‘cat’, etc. Now, do the same for the test data. You can call the resulting SFrames
image_test_cat, image_test_dog, image_test_bird, image_test_automobile

# Answer is 37.77

In [46]:
image_test_dog=image_test[image_test['label']=='dog']
image_test_cat=image_test[image_test['label']=='cat']
image_test_bird=image_test[image_test['label']=='bird']
image_test_automobile=image_test[image_test['label']=='automobile']

Finding nearest neighbors in the training set for each part of the test set: Thus far, we have queried, e.g.,
dog_model.query()
our nearest neighbors models with a single image as the input, but you can actually query with a whole set of data, and it will find the nearest neighbors for each data point. Note that the input index will be stored in the ‘query_label’ column of the output SFrame.

In [47]:
dog_query=dog_model.query(image_test_dog)
dog_query

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 128000  | 25.1473     | 813.959ms    |

| Done         | 509000  | 100         | 917.17ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,49803,33.4773536247,1
0,21235,34.4152182735,2
0,23803,34.8138642041,3
0,41752,34.9289323688,4
0,13865,37.154638537,5
1,5755,32.8458494483,1
1,38013,35.6379527896,2
1,10669,37.0042446865,3
1,11933,37.0051672517,4
1,48566,37.9279058291,5


In [48]:
dog_query[dog_query['query_label']==4]

query_label,reference_label,distance,rank
4,12089,37.4849284982,1
4,31474,39.2027543508,2
4,36093,39.21976467,3
4,542,39.5602502268,4
4,14346,39.6794815938,5


In [50]:
cat_query=cat_model.query(image_test_cat)
cat_query[cat_query['query_label']==1]

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 952.117ms    |

| Done         | 509000  | 100         | 1.13s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
1,13094,33.8680595643,1
1,10883,34.4414077553,2
1,43295,34.8348785523,3
1,6304,34.9159473893,4
1,8302,35.1367162508,5


In [55]:
bird_query=bird_model.query(image_test_bird)
bird_query[bird_query['query_label']==2]

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 119000  | 24.8954     | 936.333ms    |

| Done         | 478000  | 100         | 1.04s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
2,26768,35.8066160122,1
2,40225,35.8951138506,2
2,34669,36.152380386,3
2,16041,37.3203986512,4
2,44181,37.3771944425,5


In [57]:
automobile_query=auto_model.query(image_test_automobile)
automobile_query[automobile_query['query_label']==2]

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 128000  | 25.1473     | 964.67ms     |

| Done         | 509000  | 100         | 1.22s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
2,41471,35.7511506777,1
2,35086,35.9272861783,2
2,536,36.560166857,3
2,44415,36.7238117788,4
2,23107,36.828067959,5


Using this knowledge find the closest neighbor in to the dog test data using each of the trained models, e.g.,
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
finds 1 neighbor (that’s what k=1 does) to the dog test images (image_test_dog) in the cat portion of the training data (used to train the cat_model).
Now, do this for every combination of the labels in the training and test data.

In [58]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 1.08s        |

| Done         | 509000  | 100         | 1.22s        |

+--------------+---------+-------------+--------------+

In [59]:

dog_cat_neighbors[0]

{'distance': 36.41960343224111,
 'query_label': 0,
 'rank': 1,
 'reference_label': 33}

In [60]:
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)
dog_bird_neighbors[0]

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 120000  | 25.1046     | 738.911ms    |

| Done         | 478000  | 100         | 832.183ms    |

+--------------+---------+-------------+--------------+

{'distance': 41.753862657220424,
 'query_label': 0,
 'rank': 1,
 'reference_label': 44658}

Create an SFrame with the distances from ‘dog’ test examples to the respective nearest neighbors in each class in the training data: The ‘distance’ column in dog_cat_neighbors above contains the distance between each ‘dog’ image in the test set and its nearest ‘cat’ image in the training set. The question we want to answer is how many of the test set ‘dog’ images are closer to a ‘dog’ in the training set than to a ‘cat’, ‘automobile’ or ‘bird’. So, next we will create an SFrame containing just these distances per data point. The goal is to create an SFrame called dog_distances with 4 columns:
i. dog_distances[‘dog-dog’] ---- storing dog_dog_neighbors[‘distance’]
ii. dog_distances[‘dog-cat’] ---- storing dog_cat_neighbors[‘distance’]
iii. dog_distances[‘dog-automobile’] ---- storing dog_automobile_neighbors[‘distance’]
iv. dog_distances[‘dog-bird’] ---- storing dog_bird_neighbors[‘distance’]

In [61]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=5)
dog_bird_neighbors = bird_model.query(image_test_dog, k=5)
dog_dog_neighbors = dog_model.query(image_test_dog, k=5)
dog_automobile_neighbors = auto_model.query(image_test_dog, k=5)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 1.02s        |

| Done         | 509000  | 100         | 1.24s        |

+--------------+---------+-------------+--------------+

In [62]:
dog_distances = graphlab.SFrame({'dog-dog': dog_dog_neighbors['distance'],
                                 'dog-automobile': dog_automobile_neighbors['distance'],
                                 'dog-bird': dog_bird_neighbors['distance'],
                                 'dog-cat': dog_cat_neighbors['distance']})

In [63]:
dog_distances

dog-automobile,dog-bird,dog-cat,dog-dog
41.9579817943,41.7538626572,36.4196034322,33.4773536247
44.1437317248,41.8665825744,36.515163454,34.4152182735
44.4776165069,41.9679664705,37.0642564636,34.8138642041
45.1042453302,42.2266708142,37.1217693927,34.9289323688
45.286524911,42.3307259887,38.2837160394,37.154638537
46.0021346539,41.3382923691,38.8353256562,32.8458494483
46.3466063132,41.8627229588,38.9588642291,35.6379527896
46.7436702928,42.7126974885,39.0535346272,37.0042446865
47.7992885894,43.6588662025,39.9301894684,37.0051672517
47.9857134918,44.4454385192,40.3811562379,37.9279058291


Computing the number of correct predictions using 1-nearest neighbors for the dog class: Now that you have created the SFrame dog_distances, you will learn to use the method .apply() on this SFrame to iterate line by line and compute the number of ‘dog’ test examples where the distance to the nearest ‘dog’ was lower than that to the other classes. You will do this in three steps: i. Consider one row of the SFrame dog_distances. Let’s call this variable row. You can access each distance by calling, for example, row['dog-cat'] which, in example table above, will have value equal to 36.4196077068 for the first row. Create a function starting with def is_dog_correct(row): which returns 1 if the value for row[‘dog-dog’] is lower than that of the other columns, and 0 otherwise. That is, returns 1 if this row is correctly classified by 1-nearest neighbors, and 0 otherwise. ii. Using the function is_dog_correct(row), you can check if 1 row is correctly classified. Now, you want to count how many rows are correctly classified. You could do a for loop iterating through each row and applying the function is_dog_correct(row). This method will be really slow, because the SFrame is not optimized for this type of operation. Instead, we will use the .apply() method to iterate the function is_dog_correct for each row of the SFrame. Read about using the .apply() method here. iii. Computing the number of correct predictions for ‘dog’: You can now call: dog_distances.apply(is_dog_correct) which will return an SArray (a column of data) with a 1 for every correct row and a 0 for every incorrect one. You can call: .sum() on the result to get the total number of correctly classified ‘dog’ images in the test set!

In [64]:
def is_dog_correct(row):
    
    if row['dog-dog']<row['dog-cat'] and row['dog-dog']<row['dog-bird']  and row['dog-dog']<row['dog-automobile']:
        return (1)
    else:
        return (0)
        

dog_distances.apply(is_dog_correct).sum()

3554

Accuracy of predicting dog in the test data: Using the work you did in this question, what is the accuracy of the 1-nearest neighbor classifier at classifying ‘dog’ images from the test set?

In [65]:
3554./5000

0.7108